In [1]:
import sys
sys.path.append("../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

# Set up model and training parameters

In [3]:
params = {}

params['input_dim'] = 128
params['latent_dim'] = 3
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = False
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 1

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [4]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")
    
    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING
Epoch 0


c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_val, _, _ = autoencoder_network.define_loss(torch.tensor(train_dict['x'], dtype=torch.float32), torch.tensor(train_dict['dx'], dtype=torch.float32), params=params)
c:\Users\ianma\Desktop\Study\Python\CS4240_DeepLearning_Group41_ReproducibilityProject\Autoencoders-Reproduction\examples\../src\training.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_total_loss, train_losses, _ = network.define_loss( torch.tensor(train_dict['x'], dtype=torch.float32) , torch.tensor(train_dict['dx'], dty

Epoch 0
   Training Total Loss: 0.005688742734491825
   Training decoder Loss: 0.005075056105852127
   Training sindy_z Loss: 8.380980491638184
   Training sindy_x Loss: 6.057415962219238
   Training sindy_regularization Loss: 0.7945341467857361
   Validation Total Loss: 0.011230173520743847
   Validation decoder Loss: 0.011204272508621216
   Validation sindy_z Loss: 0.7979886531829834
   Validation sindy_x Loss: 0.17955882847309113
   Validation sindy_regularization Loss: 0.7945341467857361
Decoder Loss Ratio: 0.058202, Decoder SINDy Loss Ratio: 0.011487
Epoch 1
Epoch 1
   Training Total Loss: 0.005281791090965271
   Training decoder Loss: 0.004670129623264074
   Training sindy_z Loss: 6.845315933227539
   Training sindy_x Loss: 6.053071975708008
   Training sindy_regularization Loss: 0.6354314684867859
   Validation Total Loss: 0.009879706427454948
   Validation decoder Loss: 0.00985450204461813
   Validation sindy_z Loss: 0.6222403645515442
   Validation sindy_x Loss: 0.188499405980

KeyboardInterrupt: 